### This is a project analyzing the OneFlorida data. With the cohort of the patients who get a Thyroid ultrasound done from 2015 to 2022. 

The main part include:

1.Report the counts for the cohorts + 7.Descriptive analysis for the cohort (Exploratory Data Analysis)

2.Define the outcome variables in the regression and logistic model

3.Calculte the derivative predictors, like Geocoded social determinants of health (SVI), Charlson comorbidity index etc.

4.Conduct the Time to event analysis

5.Build the logistic model

6.Build the Linear regression model 

The outcomes are presented into tables and graphs, and are reported in word documents.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Demographic = pd.read_csv('/data2/datasets/Xingke/Thyroid_Biopsy/Raw_data/demographic_ospina_v2a.csv') 
# Procedures = pd.read_csv('/data2/datasets/Xingke/Thyroid_Biopsy/Raw_data/procedures_ospina_v2a.csv') 
# Diagnosis = pd.read_csv('/data2/datasets/Xingke/Thyroid_Biopsy/Raw_data/diagnosis_ospina_v2a.csv') 
# Encounter = pd.read_csv('/data2/datasets/Xingke/Thyroid_Biopsy/Raw_data/encounter_ospina_v2a.csv') 
# Vital = pd.read_csv('/data2/datasets/Xingke/Thyroid_Biopsy/Raw_data/vital_ospina_v2a.csv')
# Condition = pd.read_csv('/data2/datasets/Xingke/Thyroid_Biopsy/Raw_data/condition_ospina_v2a.csv') 
# address_history = pd.read_csv('/data2/datasets/Xingke/Thyroid_Biopsy/Raw_data/address_history_ospina_v2a.csv') 
# Denominator = pd.read_csv('/data2/datasets/Xingke/Thyroid_Biopsy/Adjusted.csv', dtype={'RACE': str}) 

# first_diag = pd.read_csv('/data2/datasets/Xingke/Thyroid_Biopsy/first_diag.csv')
Demographic6 = pd.read_csv('/data2/datasets/Xingke/Thyroid_Biopsy/Demographic6.csv')
Demographic6_18month = pd.read_csv('/data2/datasets/Xingke/Thyroid_Biopsy/Demographic6_18month.csv')

# 1.Report the counts for cohort

## A. Adult patients with first thyroid ultrasound + Thyroid Nodule Diagnosis 2015-2022

In [ ]:
#Get the year of biopsy
Procedures['PX_YEAR'] = Procedures['PX_DATE'].str[-4:]
# Filter the DataFrame based on biopsy CPT
filtered_Procedures = Procedures[Procedures['PX'] == '76536']

# Get the index date - fisrt ultrasound
# Based on PxDATE, if there are missing in PX date use admit date. 
filtered_Procedures['DATE'] = filtered_Procedures['PX_DATE'].fillna(filtered_Procedures['ADMIT_DATE'])
# Convert the 'Date' column to datetime type
filtered_Procedures['DATE'] = pd.to_datetime(filtered_Procedures['DATE'])
filtered_Procedures = filtered_Procedures[filtered_Procedures['DATE']>='2015-01-01 00:00:00']
print(len(filtered_Procedures['ID'].unique()))

In [ ]:
#All patient in data
print(len(Procedures['ID'].unique()))

## B. Patients with thyroid biopsy

In [ ]:
# Filter the DataFrame based on biopsy CPT
filtered_Procedures1 = Procedures[Procedures['PX'] .isin(['60100','10022','10005','10006','10004','10021','60001','60300','76942'])]

# Based on PxDATE, if there are missing in PX date use admit date. 
filtered_Procedures1['DATE'] = filtered_Procedures1['PX_DATE'].fillna(filtered_Procedures1['ADMIT_DATE'])
# Convert the 'Date' column to datetime type
filtered_Procedures1['DATE'] = pd.to_datetime(filtered_Procedures1['DATE'])
filtered_Procedures1 = filtered_Procedures1[filtered_Procedures1['DATE']>='2015-01-01 00:00:00']

Biopsy_ID = filtered_Procedures1['ID'].unique()
print(len(Biopsy_ID))

In [ ]:
# Select the first date for each ID
first_dates1 = filtered_Procedures1.groupby('ID')['DATE'].min()

# Convert the result to a DataFrame
result_df1 = pd.DataFrame({'ID': first_dates1.index, 'First_Date': first_dates1.values})
result_df1

## C. Patients with Thyroid Cancer diagnosis

In [ ]:
#US_Nodule_ID = US_Nodule['ID'].unique()
US_Nodule_all =  Diagnosis[Diagnosis['ID'] .isin(Biopsy_ID)]

#Thyroid cancer diagnosis: 193 and C73
Thyroid_cancer = US_Nodule_all[US_Nodule_all['DX'] .isin(['193','C73','C739'])]
Thyroid_cancer_ID = Thyroid_cancer['ID'].unique()
print(len(Thyroid_cancer_ID))

In [ ]:
# Get the index date - fisrt biopsy
# Based on PxDATE, if there are missing in PX date use admit date. 
Thyroid_cancer['DATE'] = Thyroid_cancer['DX_DATE'].fillna(Thyroid_cancer['ADMIT_DATE'])

# Convert the 'Date' column to datetime type
Thyroid_cancer['DATE'] = pd.to_datetime(Thyroid_cancer['DATE'])

# Select the first date for each ID
first_dates2 = Thyroid_cancer.groupby('ID')['DATE'].min()

# Convert the result to a DataFrame
result_df2 = pd.DataFrame({'ID': first_dates2.index, 'First_Date': first_dates2.values})
result_df2

# 2.Outcome variables

## First date of diagnosis

In [ ]:
# Select the first date for each ID
first_dates = filtered_Procedures.groupby('ID')['DATE'].min()

# Convert the result to a DataFrame
result_df = pd.DataFrame({'ID': first_dates.index, 'First_Date': first_dates.values})

first_diag = pd.merge(result_df,Demographic, on = 'ID',how = 'left') #Outer join? Because now all the differ date is 0 days
conditions = [
    first_diag['age']>=65,
    first_diag['age']<=39,
    first_diag['age'].isin(range(39,65)),
]
choices = ['>=65', '18-39','40-64']

first_diag['age_group'] = np.select(conditions, choices, default='Other')
first_diag['PX_YEAR'] = first_diag['First_Date'].dt.year

# Create a function to map the year and month to half-year intervals
def get_half_year(year, month):
    half_year = 'H1' if month <= 6 else 'H2'
    return f"{year}{half_year}"

# Apply the function to the 'DX_YEAR' column
first_diag['PX_YEAR_half'] = first_diag.apply(lambda row: get_half_year(row['PX_YEAR'], row['First_Date'].month), axis=1)

#Re-categorize variable
#Race
conditions = [
    first_diag['RACE'].isin(['01','04','OT']),
    first_diag['RACE'].isin(['07','NI','UN']),
    first_diag['RACE'].isin(['02']),
    first_diag['RACE'].isin(['03']),
    first_diag['RACE'].isin(['05']),
    first_diag['RACE'].isin(['06']),
]
choices = ['Other', 'Unknown','Asian','Black or African American','White','Multiple race']

first_diag['RACE1'] = np.select(conditions, choices, default='Other')

#first_diag.to_csv('first_diag.csv')
first_diag

In [ ]:
#Model 1

#Import number of adult in OneFlorida by encounter year
Total_adult = pd.read_excel('/data2/datasets/Xingke/Thyroid_Biopsy/adult_by_year.xlsx')
#The number of adults who had their first thyroid Ultrasound in a given year
unique_counts_Ultrasound = first_diag.groupby('PX_YEAR')['ID'].nunique()
unique_counts_Ultrasound = unique_counts_Ultrasound.reset_index()
#The rate of Ultrasound from 2015 to 2022
unique_counts_Ultrasound.rename(columns={'ID': 'Ultrasound_count'}, inplace=True)
#unique_counts_Ultrasound = unique_counts_Ultrasound[1:].reset_index(drop=True)
unique_counts_Ultrasound['Rate'] = unique_counts_Ultrasound['Ultrasound_count']/ Total_adult['patient_count']
unique_counts_Ultrasound

## Sequetial analysis

In [ ]:
#biopsy after ultrasound
#Index year: Ultrasound
biopsy_after = pd.merge(first_diag,first_dates1 , on = 'ID',how = 'left') #merge ultrasound date with biopsy date
biopsy_after['First_Date'] = pd.to_datetime(biopsy_after['First_Date']) #ultrasound date
biopsy_after['DATE'] = pd.to_datetime(biopsy_after['DATE']) #biopsy date
biopsy_after['differ'] = biopsy_after['DATE']-biopsy_after['First_Date']
biopsy_afterpatient = biopsy_after[biopsy_after['differ']>='0 days']
biopsy_afterpatient['Ultrasound_YEAR'] = biopsy_afterpatient['First_Date'].dt.year
biopsy_afterpatient = biopsy_afterpatient.rename(columns={'DATE': 'Biopsy date'})
print(len(biopsy_afterpatient))
unique_counts_biopsy2 = biopsy_afterpatient.groupby('Ultrasound_YEAR')['ID'].nunique()
unique_counts_biopsy2 = unique_counts_biopsy2.reset_index()
#unique_counts_biopsy2 =unique_counts_biopsy2[1:].reset_index(drop=True)
unique_counts_biopsy2['Rate'] = unique_counts_biopsy2['ID']/unique_counts_Ultrasound['Ultrasound_count']
unique_counts_biopsy2

In [ ]:
#biopsy after ultrasound
#Index year: biopsy
biopsy_after = pd.merge(first_diag,first_dates1 , on = 'ID',how = 'left') #merge ultrasound date with biopsy date
biopsy_after['First_Date'] = pd.to_datetime(biopsy_after['First_Date']) #ultrasound date
biopsy_after['DATE'] = pd.to_datetime(biopsy_after['DATE']) #biopsy date
biopsy_after['differ'] = biopsy_after['DATE']-biopsy_after['First_Date']
biopsy_afterpatient = biopsy_after[biopsy_after['differ']>='0 days']
biopsy_afterpatient['Biopsy_YEAR'] = biopsy_afterpatient['DATE'].dt.year
biopsy_afterpatient = biopsy_afterpatient.rename(columns={'DATE': 'Biopsy date'})
print(len(biopsy_afterpatient))
unique_counts_biopsy2 = biopsy_afterpatient.groupby('Biopsy_YEAR')['ID'].nunique()
unique_counts_biopsy2 = unique_counts_biopsy2.reset_index()
#unique_counts_biopsy2 =unique_counts_biopsy2[1:].reset_index(drop=True)
unique_counts_biopsy2['Rate'] = unique_counts_biopsy2['ID']/unique_counts_Ultrasound['Ultrasound_count']
unique_counts_biopsy2

In [ ]:
#cancer after biopsy
#Index year: biopsy
cancerafter = pd.merge(first_dates2,biopsy_afterpatient, on = 'ID',how = 'left')
cancerafter = cancerafter.rename(columns={'DATE': 'Cancer date'})
cancerafter['Cancer date'] = pd.to_datetime(cancerafter['Cancer date'])
cancerafter['Biopsy date'] = pd.to_datetime(cancerafter['Biopsy date'])
cancerafter['differ'] = cancerafter['Biopsy date']-cancerafter['Cancer date']
cancerafterpatient = cancerafter[cancerafter['differ']<='0 days']
cancerafterpatient['Biopsy_YEAR'] = cancerafterpatient['Biopsy date'].dt.year
print(len(cancerafterpatient))
unique_counts_cancer2 = cancerafterpatient.groupby('Biopsy_YEAR')['ID'].nunique()
unique_counts_cancer2 = unique_counts_cancer2.reset_index()
#unique_counts_cancer2['Rate'] = unique_counts_cancer2['ID']/unique_counts_biopsy['Biopsy_count']
unique_counts_cancer2

In [ ]:
#cancer after biopsy
#Index year: cancer
cancerafter = pd.merge(first_dates2,biopsy_afterpatient, on = 'ID',how = 'left')
cancerafter = cancerafter.rename(columns={'DATE': 'Cancer date'})
cancerafter['Cancer date'] = pd.to_datetime(cancerafter['Cancer date'])
cancerafter['Biopsy date'] = pd.to_datetime(cancerafter['Biopsy date'])
cancerafter['differ'] = cancerafter['Biopsy date']-cancerafter['Cancer date']
cancerafterpatient = cancerafter[cancerafter['differ']<='0 days']
cancerafterpatient['Cancer_YEAR'] = cancerafterpatient['Cancer date'].dt.year
print(len(cancerafterpatient))
unique_counts_cancer2 = cancerafterpatient.groupby('Cancer_YEAR')['ID'].nunique()
unique_counts_cancer2 = unique_counts_cancer2.reset_index()
#unique_counts_cancer2['Rate'] = unique_counts_cancer2['ID']/unique_counts_biopsy['Biopsy_count']
unique_counts_cancer2

In [ ]:
biopsy_afterID = biopsy_afterpatient['ID'].unique()
cancer_afterID = cancerafterpatient['ID'].unique()

In [ ]:
#After runing the model
Biopsy_18month = Demographic6_18month[Demographic6_18month['Biopsy_18month']  == 1]
Biopsy_18monthID = Biopsy_18month['ID'].unique()
Biopsy_18monthpt = biopsy_afterpatient[biopsy_afterpatient['ID'].isin(Biopsy_18monthID)]

Biopsy_18monthpt['Ultrasound_YEAR'] = Biopsy_18monthpt['First_Date'].dt.year
Biopsy_18monthpt = Biopsy_18monthpt.rename(columns={'DATE': 'Biopsy date'})
unique_counts_biopsy3 = Biopsy_18monthpt.groupby('Ultrasound_YEAR')['ID'].nunique()
unique_counts_biopsy3 = unique_counts_biopsy3.reset_index()
#unique_counts_biopsy3 =unique_counts_biopsy3[1:].reset_index(drop=True)
unique_counts_biopsy3['Rate'] = unique_counts_biopsy3['ID']/unique_counts_Ultrasound['Ultrasound_count']
unique_counts_biopsy3

In [ ]:
#After runing the model
Biopsy_18month = Demographic6_18month1[Demographic6_18month1['Cancer_18month']  == 1]
Biopsy_18monthID = Biopsy_18month['ID'].unique()
Biopsy_18monthpt = biopsy_afterpatient[biopsy_afterpatient['ID'].isin(Biopsy_18monthID)]

Biopsy_18monthpt['Biopsy_YEAR'] = Biopsy_18monthpt['Biopsy date'].dt.year
Biopsy_18monthpt = Biopsy_18monthpt.rename(columns={'DATE': 'Biopsy date'})
unique_counts_cancer3 = Biopsy_18monthpt.groupby('Biopsy_YEAR')['ID'].nunique()
unique_counts_cancer3 = unique_counts_cancer3.reset_index()
#unique_counts_cancer3 =unique_counts_cancer3[1:].reset_index(drop=True)
unique_counts_cancer3['Rate'] = unique_counts_cancer3['ID']/unique_counts_Ultrasound['Ultrasound_count']
unique_counts_cancer3

# 3.Variables of interest

## Geocoded social determinants of health (SVI)

In [ ]:
#Get the Rurality of each zip code
RUCA = pd.read_excel('RUCA2010zipcode.xlsx', sheet_name=1,dtype={0:float}) 
Rurality =  pd.merge(Encounter, RUCA, left_on = 'FACILITY_LOCATION', right_on = 'ZIP_CODE',how = 'left')
Rurality = Rurality.drop(columns=['ZIP_CODE','STATE','ZIP_TYPE'])

#Get the SVI of each county
SVI = pd.read_csv('SVI_2020_US_county.csv') 
#Convert county to zipcode
COUNTY_ZIP = pd.read_excel('ZIP_COUNTY_032023.xlsx')
SVI2 = pd.merge(SVI,COUNTY_ZIP,left_on = 'STCNTY',right_on = 'COUNTY',how = 'left')
SVI3 = SVI2[['ZIP','RPL_THEMES']].drop_duplicates(subset='ZIP', keep='first')

#Merge Rurality and SVI
Rurality_SVI = pd.merge(Rurality, SVI3, left_on = 'FACILITY_LOCATION', right_on = 'ZIP',how = 'left')
Rurality_SVI = Rurality_SVI.drop(columns=['ZIP'])

filtered_Procedures_new = filtered_Procedures[['ID', 'ENCOUNTERID','ADMIT_DATE','PX_DATE', 'PX' ]]
SVI1 = pd.merge(Rurality_SVI, filtered_Procedures_new, on = 'ENCOUNTERID',how = 'left')
SVI2 = SVI1[SVI1['PX'] == '76536' ]
SVI2.rename(columns={'ID_x': 'ID'}, inplace=True)

In [ ]:
#Personal zip code
Personzip =  pd.merge(address_history, RUCA, left_on = 'ADDRESS_ZIP5', right_on = 'ZIP_CODE',how = 'left')
Personzip = Personzip.drop(columns=['ZIP_CODE','STATE','ZIP_TYPE'])
Personzip = pd.merge(Personzip, SVI3, left_on = 'ADDRESS_ZIP5', right_on = 'ZIP',how = 'left')
Personzip = Personzip.drop(columns=['ZIP'])
Personzip

## Insurance

In [ ]:
# Define the lambda function to assign values based on conditions in column PAYER_TYPE_PRIMARY
def assign_value(row):
    row = str(row['PAYER_TYPE_PRIMARY'])
    if row.startswith('1'):
        return 'MEDICARE'
    elif row.startswith('2'):
        return 'MEDICAID'
    elif row.startswith('3') or row.startswith('4'):
        return 'OtherGOV'
    elif row.startswith('5') or row.startswith('6'):
        return 'PRIVATE'
    elif row.startswith('8'):
        return 'NOPAY'
    elif row.startswith('7') or row.startswith('9') and row != '9999':
        return 'Other'
    elif row == 'OT':
        return 'Other'
    elif row == '9999':
        return 'Unknown'
    else:
        return 'Unknown'

# Apply the lambda function to assign values in column B
SVI2['Insurance'] = SVI2.apply(assign_value, axis=1)

SVI2

In [ ]:
#For facility zip
#Merge with first date of biopsy 
Encounter1 = pd.merge(result_df,SVI2,on = 'ID',how = 'left')
Encounter1['ADMIT_DATE_x'] = pd.to_datetime(Encounter1['ADMIT_DATE_x'])

#Find the Encounter date of the most recent date
Encounter1['differ'] = Encounter1['ADMIT_DATE_x'] - Encounter1['First_Date']
#Encounter1['differ2'] = Encounter1['differ'].abs()
#Encounter2 =  Encounter1.groupby('ID')['differ'].min().reset_index().merge(Encounter1, on=['ID', 'differ2'])

# Perform the groupby operation to get the minimum value for each ID
min_values = Encounter1.groupby('ID')['differ'].min().reset_index()
Encounter2 = min_values.merge(Encounter1, on=['ID', 'differ'], how='inner')
Encounter22 = Encounter2[Encounter2['differ']>='-180 days +21:00:00']

#right join???
Encounter3 = Encounter22[['ID','Insurance']].drop_duplicates(subset='ID',keep='first')
print(len(Encounter3 ['ID'].unique()))

Encounter4 = Encounter22[['ID','RUCA1','RUCA2','FACILITY_LOCATION']].drop_duplicates(subset='ID', keep='first')
print(len(Encounter4 ['ID'].unique()))

Encounter6 = Encounter22[['ID','RPL_THEMES']].drop_duplicates(subset='ID', keep='first')
print(len(Encounter6 ['ID'].unique()))

In [ ]:
#For personal zip
#Merge with first date of biopsy 
Personzip1 = pd.merge(result_df,Personzip,on = 'ID',how = 'left')
Personzip1['ADDRESS_PERIOD_START'] = pd.to_datetime(Personzip1['ADDRESS_PERIOD_START'])

#Find the Encounter date of the most recent date
Personzip1['differ'] = Personzip1['ADDRESS_PERIOD_START'] - Personzip1['First_Date']
Personzip1['differ2'] = Personzip1['differ'].abs()
Personzip2 =  Personzip1.groupby('ID')['differ2'].min().reset_index().merge(Personzip1, on=['ID', 'differ2'])

#right join???
Personzip3 = Personzip2[['ID','RPL_THEMES','ADDRESS_ZIP5']].drop_duplicates(subset='ID',keep='first')
print(len(Personzip3 ['ID'].unique()))

Personzip4 = Personzip2[['ID','RUCA1']].drop_duplicates(subset='ID',keep='first')
print(len(Personzip4 ['ID'].unique()))

Personzip4.rename(columns={'RUCA1': 'RUCA1_Person'}, inplace=True)
Personzip3.rename(columns={'RPL_THEMES': 'SVI_Person'}, inplace=True)

## Charlson comorbidity index

In [ ]:
#Used Packages from Shuang to calculate CCI index
Commoridity = pd.read_csv('commoridity.csv')
Commoridity['CCI'] = Commoridity['AIDS']*6 + Commoridity['Any malignancy,includes leukemia and lympyoma']*2 +Commoridity['Cerebrovascular Disease']*1 +Commoridity['Chronic Pulmonary Disease']*1 +Commoridity['Congestive Heart Failure']*1 +Commoridity['Dementia']*1 +Commoridity['Diabetes']*1 +Commoridity['Diabetes with Chronic Complications']*2 +Commoridity['Hemiplegia or Paraplegia']*2 +Commoridity['Metastaic solid tumor']*6 +Commoridity['Mild Liver Disease']*1 +Commoridity['Moderate or Severe Liver Disease']*3 +Commoridity['Myocardial Infarction']*1 +Commoridity['Peptic Ulcer Disease']*1+Commoridity['Peripheral Vascular Disease']*1+Commoridity['Renal Disease']*2 +Commoridity['Rheumatologic Disease']*1
first_diag = pd.merge(first_diag,Commoridity,left_on = 'ID',right_on = 'PATID', how = 'left')
first_diag['CCI'] = first_diag['CCI'].fillna(0)

## BMI
At the time of biopsy; if missing then on the most recent visit.

In [ ]:
#Clean the data for vital
Vital1 = Vital[['ID','MEASURE_DATE','ORIGINAL_BMI']]
Vital2 = Vital1.drop_duplicates()
Vital3 = Vital2.dropna(subset=['ORIGINAL_BMI'])
Vital3['First_Date'] = pd.to_datetime(Vital3['MEASURE_DATE'])

#Merge with first date of biopsy 
BMI1 = pd.merge(result_df,Vital3,on = 'ID',how = 'left')
BMI2 = BMI1.dropna(subset=['ORIGINAL_BMI'])

#Find the BMI of the most recent date
BMI2['differ'] = BMI2['First_Date_y'] - BMI2['First_Date_x']
BMI2['differ2'] = BMI2['differ'].abs()
first_date_rows =  BMI2.groupby('ID')['differ2'].min().reset_index().merge(BMI2, on=['ID', 'differ2'])
#right join???

#There are duplicate IDs, that is because the same day has multiple BMIs, so let's keep the first one
print(len(first_date_rows['ID'].unique()))
first_date_rows1 = first_date_rows.drop_duplicates(subset='ID', keep='first')
first_date_rows1

In [ ]:
conditions = [
    first_date_rows1['ORIGINAL_BMI']<18.5,
    (first_date_rows1['ORIGINAL_BMI'] >= 18.5) & (first_date_rows1['ORIGINAL_BMI'] < 25),
    (first_date_rows1['ORIGINAL_BMI'] >= 25) & (first_date_rows1['ORIGINAL_BMI'] < 30),
    (first_date_rows1['ORIGINAL_BMI'] >= 30) & (first_date_rows1['ORIGINAL_BMI'] < 40),
    first_date_rows1['ORIGINAL_BMI']>= 40
]

choices = ['underweight', 'healthy range','overweight','obesity','severe obesity']

first_date_rows1['BMI_group'] = np.select(conditions, choices, default='UN')

first_date_rows1['BMI_group'].value_counts()

# Merge all the Table

In [ ]:
#Merge with BMI
Demographic1 = pd.merge(first_diag,first_date_rows1,on = 'ID',how = 'left')
Demographic2 = pd.merge(Demographic1,Personzip3,on = 'ID',how = 'left')
Demographic2 = pd.merge(Demographic2,Personzip4,on = 'ID',how = 'left')
# Demographic2 = Demographic2[['ID','SEX','age','HISPANIC','RACE','CCI','SVI_Person','RUCA1_Person']]

#Merge with Insurance
Demographic3 = pd.merge(Demographic2,Encounter3,on = 'ID',how = 'left')

#Merge with rurality
Demographic5 = pd.merge(Demographic3,Encounter4,on = 'ID',how = 'left')

#Re-categorize variable
#Race
conditions = [
    Demographic5['RACE'].isin(['01','04','OT']),
    Demographic5['RACE'].isin(['07','NI','UN']),
    Demographic5['RACE'].isin(['02']),
    Demographic5['RACE'].isin(['03']),
    Demographic5['RACE'].isin(['05']),
    Demographic5['RACE'].isin(['06']),
]

choices = ['Other', 'Unknown','Asian','Black or African American','White','Multiple race']

Demographic5['RACE1'] = np.select(conditions, choices, default='Other')

#HISPANIC
conditions = [
    Demographic5['HISPANIC'].isin(['Y']),
    Demographic5['HISPANIC'].isin(['R','NI','UN']),
    Demographic5['HISPANIC'].isin(['N']),
]

choices = ['Yes', 'Unknown','No']

Demographic5['HISPANIC1'] = np.select(conditions, choices, default='Other')

#RUCA
#https://www.hrsa.gov/rural-health/about-us/what-is-rural
conditions = [
    Demographic5['RUCA1'].isin([1.0,2.0,3.0]),
    Demographic5['RUCA1'].isin([4.0,5.0,6.0,7.0,8.0,9.0,10.0]),
]

choices = ['Urban', 'Rural']

Demographic5['RUCA'] = np.select(conditions, choices, default='UN')

#Age
conditions = [
    (Demographic5['age'] >= 18) & (Demographic5['age'] < 40),
    (Demographic5['age'] >= 40) & (Demographic5['age'] <= 65),
    Demographic5['age']>65
]

choices = ['18-39', '40-64','>65']

Demographic5['age_group'] = np.select(conditions, choices, default='UN')

#CCI
conditions = [
    Demographic5['CCI'] == 0,
    Demographic5['CCI'].isin([1.0,2.0]),
    Demographic5['CCI'].isin([3.0,4.0]),
    Demographic5['age']>=5.0
]

choices = ['None', 'Mild','Moderate','Severe']

Demographic5['CCI1'] = np.select(conditions, choices, default='UN')

#RUCA_person
conditions = [
    Demographic5['RUCA1_Person'].isin([1.0,2.0,3.0]),
    Demographic5['RUCA1_Person'].isin([4.0,5.0,6.0,7.0,8.0,9.0,10.0]),
]

choices = ['Urban', 'Rural']

Demographic5['RUCA_Person'] = np.select(conditions, choices, default='UN')


#SVI
conditions = [
    Demographic5['SVI_Person']<=0.5,
    (Demographic5['SVI_Person'] > 0.50) & (Demographic5['SVI_Person'] <= 0.75),
    Demographic5['SVI_Person']>0.75
]

choices = ['<0.5', '0.5-0.75','>0.75']

Demographic5['SVI_Person2'] = np.select(conditions, choices, default='UN')

In [ ]:
#Merge with SVI
Demographic6 = pd.merge(Demographic5,Encounter6,on = 'ID',how = 'left')

# Function to check if value is in list A
def check_presence(value):
    return 1 if value in cancer_afterID else 0

# Apply the function to create column B
Demographic6['Cancer'] = Demographic6['ID'].apply(lambda x: check_presence(x))

#SVI
conditions = [
    Demographic6['RPL_THEMES']<=0.5,
    (Demographic6['RPL_THEMES'] > 0.50) & (Demographic6['RPL_THEMES'] <= 0.75),
    Demographic6['RPL_THEMES']>0.75
]

choices = ['<0.5', '0.5-0.75','>0.75']

Demographic6['SVI'] = np.select(conditions, choices, default='UN')

# #Must delete them to avoid multi-colinear
# # Delete rows where value in sex is 'UN' (Only 1 people)
# Demographic6 = Demographic6.loc[Demographic5['SEX'] != 'UN']
# Demographic6 = Demographic6.loc[Demographic6['SVI'] != 'UN'] #only 4 people

# #SVI will not be in the model as category, don't delete
# Demographic6 = Demographic6.loc[Demographic6['RUCA_Person'] != 'UN']
# #Demographic6.to_csv('Demographic6.csv')

# 4. Time to cancer analysis

## Ultrasound to biopsy

In [ ]:
biopsy_afterpatient1 = biopsy_afterpatient[['ID','Biopsy date']]

first_diag2 =  pd.merge(first_diag,biopsy_afterpatient1, on = 'ID',how = 'left')
first_diag2['Biopsy date'] = first_diag2['Biopsy date'].fillna('2022-06-30')
#Calculate time to biopsy from first ultrasound
first_diag2['Time to survival']  = first_diag2['Biopsy date'] - first_diag2['First_Date']
#Make the columns numeric
first_diag2['Time to survival1']  = first_diag2['Time to survival'] / pd.Timedelta(days=1)
first_diag2['Time to survival2'] = first_diag2['Time to survival'] / pd.Timedelta(days=365.25)

# Function to check if value is in list A
def check_presence(value):
    return 1 if value in biopsy_afterID else 0

# Apply the function to create column B
first_diag2['Biopsy'] = first_diag2['ID'].apply(lambda x: check_presence(x))

plt.hist(first_diag2['Time to survival2'], bins = 20)
plt.show()

In [ ]:
#For all the patients
from lifelines import KaplanMeierFitter
kmf = KaplanMeierFitter() 
kmf.fit(durations = first_diag2['Time to survival2'], event_observed = first_diag2['Biopsy']) 
kmf.plot_survival_function()

# Add x-label
plt.xlabel('Time (in years)')

plt.show()


## Biopsy to cancer

In [ ]:
cancerafterpatient1 = cancerafterpatient[['ID','Cancer date']]
biopsy_afterpatient =  pd.merge(biopsy_afterpatient,cancerafterpatient1, on = 'ID',how = 'left')
biopsy_afterpatient['Cancer date'] = biopsy_afterpatient['Cancer date'].fillna('2022-06-30')
#Calculate time to cancer from first biopsy 
biopsy_afterpatient['Time to survival']  = biopsy_afterpatient['Cancer date'] - biopsy_afterpatient['Biopsy date']
#Make the columns numeric
biopsy_afterpatient['Time to survival1']  = biopsy_afterpatient['Time to survival'] / pd.Timedelta(days=1)
biopsy_afterpatient['Time to survival2'] = biopsy_afterpatient['Time to survival'] / pd.Timedelta(days=365.25)

# Function to check if value is in list A
def check_presence(value):
    return 1 if value in cancer_afterID else 0

# Apply the function to create column B
biopsy_afterpatient['Cancer'] = biopsy_afterpatient['ID'].apply(lambda x: check_presence(x))

plt.hist(biopsy_afterpatient['Time to survival2'], bins = 20)
plt.show()

In [ ]:
from lifelines import KaplanMeierFitter
kmf = KaplanMeierFitter() 
kmf.fit(durations = biopsy_afterpatient['Time to survival2'], event_observed = biopsy_afterpatient['Cancer']) 
kmf.plot_survival_function()

# Add x-label
plt.xlabel('Time (in years)')

plt.show()


## 18 month cut off

In [ ]:
biopsy_afterpatient1 = biopsy_afterpatient[['ID','Biopsy date']]

first_diag2 =  pd.merge(biopsy_afterpatient1,first_diag, on = 'ID',how = 'left')
#Calculate time to biopsy from first ultrasound
first_diag2['Time to survival']  = first_diag2['Biopsy date'] - first_diag2['First_Date']
#Make the columns numeric
first_diag2['Time to survival1']  = first_diag2['Time to survival'] / pd.Timedelta(days=1)
first_diag2['Time to survival2'] = first_diag2['Time to survival'] / pd.Timedelta(days=365.25)

def check_presence(value):
    return 1 if value < 548 else 0
first_diag2['Biopsy_18month'] = first_diag2['Time to survival1'].apply(check_presence)
first_diag2 = first_diag2[['ID','Time to survival1','Biopsy_18month']]
first_diag2 = first_diag2.rename(columns={'Time to survival1': 'Time to survival_biopsy'})

In [ ]:
first_diag3 =  pd.merge(first_diag,first_diag2, on = 'ID',how = 'left')
first_diag3['Time to survival_biopsy'] = first_diag3['Time to survival_biopsy'].fillna(548.0)
first_diag3['Biopsy_18month'] = first_diag3['Biopsy_18month'].fillna(0)

from lifelines import KaplanMeierFitter
kmf = KaplanMeierFitter() 
kmf.fit(durations = first_diag3['Time to survival_biopsy'], event_observed = first_diag3['Biopsy_18month']) 
kmf.plot_survival_function()

# Add x-label
plt.xlabel('Time (in days)')
plt.xlim(-20, 548)

plt.show()

In [ ]:
first_diag_withcancer2 = biopsy_afterpatient[biopsy_afterpatient['ID'].isin(cancer_afterID)]
first_diag_withcancer2['Cancer_18month'] = first_diag_withcancer2['Time to survival1'].apply(check_presence)
first_diag_withcancer2 = first_diag_withcancer2[['ID','Time to survival1','Cancer_18month']]
first_diag_withcancer2 = first_diag_withcancer2.rename(columns={'Time to survival1': 'Time to survival_cancer'})

In [ ]:
first_diag3 =  pd.merge(biopsy_afterpatient,first_diag_withcancer2, on = 'ID',how = 'left')
first_diag3['Time to survival_cancer'] = first_diag3['Time to survival_cancer'].fillna(548.0)
first_diag3['Cancer_18month'] = first_diag3['Cancer_18month'].fillna(0)

from lifelines import KaplanMeierFitter
kmf = KaplanMeierFitter() 
kmf.fit(durations = first_diag3['Time to survival_cancer'], event_observed = first_diag3['Cancer_18month']) 
kmf.plot_survival_function()

# Add x-label
plt.xlabel('Time (in days)')
plt.xlim(-20, 548)

plt.show()

In [ ]:
Demographic6_18month1 = pd.merge(Demographic6,first_diag_withcancer2, on = 'ID',how = 'left')
Demographic6_18month1['Cancer_18month'] = Demographic6_18month1['Cancer_18month'].fillna(0)
Demographic6_18month = pd.merge(Demographic6_18month1,first_diag2, on = 'ID',how = 'left')
Demographic6_18month['Biopsy_18month'] = Demographic6_18month['Biopsy_18month'].fillna(0)
print(len(Demographic6_18month1[Demographic6_18month1['Cancer_18month']  == 1]))
print(len(Demographic6_18month[Demographic6_18month['Biopsy_18month']  == 1]))

# 5. Build the logistic model for aim 2

## Ultrasound to Biopsy

### Multivariate

In [ ]:
Demographic6_18month = pd.read_csv('/data2/datasets/Xingke/Thyroid_Biopsy/Demographic6_18month.csv')

In [ ]:
Demographic6_18month1 = Demographic6_18month 

In [ ]:
Demographic6_18month = Demographic6_18month.loc[Demographic6_18month['SEX'] != 'UN']
Demographic6_18month = Demographic6_18month.loc[Demographic6_18month['Insurance'] != 'Other']
Demographic6_18month = Demographic6_18month.loc[Demographic6_18month['RUCA_Person'] != 'UN']

Demographic_final = Demographic6_18month.drop(columns=['Unnamed: 0','Unnamed: 0.1','Time to survival_cancer','Time to survival_biopsy', 'Cancer_18month','Cancer','ID','RACE','HISPANIC','RUCA2','RUCA1','SVI','age_group','CCI','SVI_Person2','RUCA1_Person'])

Demographic_final['RPL_THEMES1'] = pd.qcut(Demographic_final['RPL_THEMES'], q=4, labels=False)
Demographic_final['SVI_Person1'] = pd.qcut(Demographic_final['SVI_Person'], q=4, labels=False)

In [ ]:
# Demographic6_18month = Demographic6 #VIF

#Delete rows where value in sex is 'UN' (Only 1 people)

Demographic6_18month = Demographic6_18month.loc[Demographic6_18month['SEX'] != 'UN']
Demographic6_18month = Demographic6_18month.loc[Demographic6_18month['Insurance'] != 'Other']
Demographic6_18month = Demographic6_18month.loc[Demographic6_18month['RUCA_Person'] != 'UN']

Demographic_final = Demographic6_18month.drop(columns=['Unnamed: 0','Unnamed: 0.1','Time to survival_cancer','Time to survival_biopsy', 'Cancer_18month','Cancer','ID','RACE','HISPANIC','RUCA2','RUCA1','SVI','age_group','CCI','SVI_Person2','RUCA1_Person'])

Demographic_final['RPL_THEMES1'] = pd.qcut(Demographic_final['RPL_THEMES'], q=4, labels=False)
Demographic_final['SVI_Person1'] = pd.qcut(Demographic_final['SVI_Person'], q=4, labels=False)

Demographic_final = Demographic_final.drop(columns=['RPL_THEMES','SVI_Person','RUCA_Person','RUCA'])

Demographic_final = Demographic_final.dropna()

#One hot encoding for categorical variables
Dummy = ['HISPANIC1','RACE1','SEX','Insurance','CCI1','RPL_THEMES1','SVI_Person1']


def convert_to_dummies(dataframe, variables):
    for variable in variables:
        if  variable == 'Insurance':
            dummies = pd.get_dummies(dataframe[variable], prefix=variable)
            dummies = dummies.drop(columns=[f'{variable}_MEDICAID'])
        elif variable == 'RACE1':
            dummies = pd.get_dummies(dataframe[variable], prefix=variable)
            dummies = dummies.drop(columns=[f'{variable}_White'])
        elif variable == 'CCI1':
            dummies = pd.get_dummies(dataframe[variable], prefix=variable)
            dummies = dummies.drop(columns=[f'{variable}_None'])
        elif variable == 'RPL_THEMES1':
            dummies = pd.get_dummies(dataframe[variable], prefix=variable)
            dummies = dummies.drop(columns=[f'{variable}_3.0'])
        elif variable == 'SVI_Person1':
            dummies = pd.get_dummies(dataframe[variable], prefix=variable)
            dummies = dummies.drop(columns=[f'{variable}_3.0'])
        elif variable == 'age':
            continue  # Skip further processing for these variables
        else:
            dummies = pd.get_dummies(dataframe[variable], prefix=variable, drop_first=True)
        
        if variable != 'age':
            dummies = dummies.astype(int)
            dataframe = pd.concat([dataframe, dummies], axis=1)
            dataframe.drop(columns=[variable], inplace=True)
    
    return dataframe

#VIF
# Demographic_final = Demographic6_18month.drop(columns=['Unnamed: 0','ID','RACE','HISPANIC','RUCA2','RUCA1','SVI','age_group','CCI','SVI_Person2','RUCA1_Person'])

Demographic_final_dummy = convert_to_dummies(Demographic_final, Dummy)
Demographic_final_dummy = Demographic_final_dummy.dropna()
Demographic_final_dummy

In [ ]:
Demographic7 = Demographic6_18month[['HISPANIC1','RACE1','SEX','Insurance','RUCA','SVI','age_group','CCI1','RUCA_Person','SVI_Person2']]

# Create an empty list to store tuples of (variable, level, count)
value_counts_list = []

# Iterate through each variable and calculate value_counts
for col in Demographic7:
    value_counts = pd.Series(Demographic7[col]).value_counts()
    for index, count in value_counts.items():
        value_counts_list.append((col, index, count))

# Convert the list of tuples into a DataFrame
value_counts_df = pd.DataFrame(value_counts_list, columns=['Variable', 'Level', 'Count'])
value_counts_df['Percent'] = value_counts_df['Count']/67238

value_counts_df

In [ ]:
#without Rurality
import seaborn as sns
import statsmodels.formula.api as smf
from statsmodels.tools import add_constant
import statsmodels.api as sm
import researchpy as rp

# Split the data into training and testing sets
X = Demographic_final_dummy.drop(columns=['Biopsy_18month'])  # Input features
y = Demographic_final_dummy['Biopsy_18month']
#X = add_constant(X)

# Fit logistic regression model
model = sm.Logit(y, X)
result = model.fit()

# Convert summary to a DataFrame
summary_df = pd.DataFrame(result.summary().tables[1])

# Save the summary DataFrame to a CSV file
# summary_df.to_csv('logistic_model_summary_test.csv', index=False)

# Display model summary
print(result.summary())

In [ ]:
def create_variable_dataframes(df, excel_filename):
    # Get the column names
    column_names = df.columns.tolist()
    column_names.remove('Biopsy_18month')
    
    # Create an empty DataFrame to store results
    result_df = pd.DataFrame()
    
    # Iterate through the variables of interest
    for column_name in column_names:
        # Create a new dataframe for the variable with the event and time-to-survival columns
        variable_df = pd.DataFrame({
            column_name: df[column_name],
            'Biopsy_18month': df['Biopsy_18month']
        })
        
        # Convert specific variables to dummy variables while dropping categories
        variables_to_convert = [column_name]  # Add more variables if needed
        Demographic_final_dummy = convert_to_dummies(variable_df, variables_to_convert)
        Demographic_final_dummy = Demographic_final_dummy.dropna()

        # Split the data into training and testing sets
        X = Demographic_final_dummy.drop(columns=['Biopsy_18month']) 
        y = Demographic_final_dummy['Biopsy_18month']
        X = add_constant(X)

        # Fit logistic regression model
        model = sm.Logit(y, X)
        result = model.fit()

        # Store model results in the master DataFrame
        model_summary = pd.DataFrame(result.summary().tables[1])
        model_summary['Variable'] = column_name  # Add a column for the variable name
        result_df = pd.concat([result_df, model_summary])
    
    # Save the master DataFrame to an Excel file
    result_df.to_excel(excel_filename, index=False)

# Call the function to create the single DataFrame with all model results
create_variable_dataframes(Demographic_final, 'logistic1.xlsx')


## Biopsy to cancer

### Multivariate

In [ ]:
Demographic6_18month2 = Demographic6_18month[Demographic6_18month['ID'].isin(biopsy_afterID)]
Demographic6_18month2 = Demographic6_18month2.loc[Demographic6_18month2['RUCA_Person'] != 'UN']
Demographic_final = Demographic6_18month2.drop(columns=['Unnamed: 0','Unnamed: 0.1','Time to survival_cancer','Time to survival_biopsy','Cancer','Biopsy_18month','ID','RACE','HISPANIC','RUCA2','RUCA1','SVI','age_group','CCI','SVI_Person2','RUCA1_Person'])

Demographic_final['RPL_THEMES1'] = pd.qcut(Demographic_final['RPL_THEMES'], q=4, labels=False)
Demographic_final['SVI_Person1'] = pd.qcut(Demographic_final['SVI_Person'], q=4, labels=False)

Demographic_final = Demographic_final.drop(columns=['RPL_THEMES','SVI_Person','RUCA_Person','RUCA'])

Demographic_final = Demographic_final.dropna()

#VIF
# Demographic_final = Demographic6_18month.drop(columns=['Unnamed: 0','ID','RACE','HISPANIC','RUCA2','RUCA1','SVI','age_group','CCI','SVI_Person2','RUCA1_Person'])

def convert_to_dummies(dataframe, variables):
    for variable in variables:
        if  variable == 'Insurance':
            dummies = pd.get_dummies(dataframe[variable], prefix=variable)
            dummies = dummies.drop(columns=[f'{variable}_MEDICAID'])
        elif variable == 'RACE1':
            dummies = pd.get_dummies(dataframe[variable], prefix=variable)
            dummies = dummies.drop(columns=[f'{variable}_White'])
        elif variable == 'CCI1':
            dummies = pd.get_dummies(dataframe[variable], prefix=variable)
            dummies = dummies.drop(columns=[f'{variable}_None'])
        elif variable == 'RPL_THEMES1':
            dummies = pd.get_dummies(dataframe[variable], prefix=variable)
            dummies = dummies.drop(columns=[f'{variable}_3'])
        elif variable == 'SVI_Person1':
            dummies = pd.get_dummies(dataframe[variable], prefix=variable)
            dummies = dummies.drop(columns=[f'{variable}_3.0'])
        elif variable == 'age':
            continue  # Skip further processing for these variables
        else:
            dummies = pd.get_dummies(dataframe[variable], prefix=variable, drop_first=True)
        
        if variable != 'age':
            dummies = dummies.astype(int)
            dataframe = pd.concat([dataframe, dummies], axis=1)
            dataframe.drop(columns=[variable], inplace=True)
    
    return dataframe

Demographic_final_dummy = convert_to_dummies(Demographic_final, Dummy)
Demographic_final_dummy = Demographic_final_dummy.dropna()

Demographic7 = Demographic6_18month2[['HISPANIC1','RACE1','SEX','Insurance','RUCA','SVI','age_group','CCI1','RUCA_Person','SVI_Person2']]

# Create an empty list to store tuples of (variable, level, count)
value_counts_list = []

# Iterate through each variable and calculate value_counts
for col in Demographic7:
    value_counts = pd.Series(Demographic7[col]).value_counts()
    for index, count in value_counts.items():
        value_counts_list.append((col, index, count))

# Convert the list of tuples into a DataFrame
value_counts_df = pd.DataFrame(value_counts_list, columns=['Variable', 'Level', 'Count'])
value_counts_df['Percent'] = value_counts_df['Count']/67238

#value_counts_df.to_csv('value_counts.csv')
value_counts_df

Demographic_final_dummy

In [ ]:
# Split the data into training and testing sets
X = Demographic_final_dummy.drop(columns=['Cancer_18month'])  # Input features
y = Demographic_final_dummy['Cancer_18month']
#X = add_constant(X)

# Fit logistic regression model
model = sm.Logit(y, X)
result = model.fit()

# Convert summary to a DataFrame
summary_df = pd.DataFrame(result.summary().tables[1])

# Save the summary DataFrame to a CSV file
# summary_df.to_csv('logistic_model_summary2.csv', index=False)

# Display model summary
print(result.summary())

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Assuming 'X' and 'y' have been defined as before:
X = sm.add_constant(X)  # Adding a constant is important for VIF calculation
model = sm.Logit(y, X)
result = model.fit()

# Now, get the covariance matrix from the fitted results
cov = result.cov_params()

# Calculate the correlation matrix
corr = cov / np.outer(result.bse, result.bse)

# Inverting the correlation matrix gives us variance inflation factors on the diagonal
vif = np.diag(np.linalg.inv(corr))

# We can put this into a DataFrame for a better presentation
vif_df = pd.DataFrame({
    'Variable': X.columns,
    'VIF': vif
})

print(vif_df)


In [ ]:
def create_variable_dataframes(df, excel_filename):
    # Get the column names
    column_names = df.columns.tolist()
    column_names.remove('Cancer_18month')
    
    # Create an empty DataFrame to store results
    result_df = pd.DataFrame()
    
    # Iterate through the variables of interest
    for column_name in column_names:
        # Create a new dataframe for the variable with the event and time-to-survival columns
        variable_df = pd.DataFrame({
            column_name: df[column_name],
            'Cancer_18month': df['Cancer_18month']
        })
        
        # Convert specific variables to dummy variables while dropping categories
        variables_to_convert = [column_name]  # Add more variables if needed
        Demographic_final_dummy = convert_to_dummies(variable_df, variables_to_convert)
        Demographic_final_dummy = Demographic_final_dummy.dropna()

        # Split the data into training and testing sets
        X = Demographic_final_dummy.drop(columns=['Cancer_18month']) 
        y = Demographic_final_dummy['Cancer_18month']
        X = add_constant(X)

        # Fit logistic regression model
        model = sm.Logit(y, X)
        result = model.fit()

        # Store model results in the master DataFrame
        model_summary = pd.DataFrame(result.summary().tables[1])
        model_summary['Variable'] = column_name  # Add a column for the variable name
        result_df = pd.concat([result_df, model_summary])
    
    # Save the master DataFrame to an Excel file
    result_df.to_excel(excel_filename, index=False)

# Call the function to create the single DataFrame with all model results
create_variable_dataframes(Demographic_final, 'logistic2.xlsx')

# 6. Build the Linear regression model for aim 2

## Ultrasound to biopsy

In [ ]:
Demographic6_18month = pd.read_csv('/data2/datasets/Xingke/Thyroid_Biopsy/Demographic6_18month.csv')
Demographic6_18month = Demographic6_18month.drop(columns=['Unnamed: 0','Unnamed: 0.1'])
Demographic6_18month1 = Demographic6_18month

Demographic6_biopsy = Demographic6_18month[Demographic6_18month['Biopsy_18month']  == 1]
Demographic_final = Demographic6_biopsy.drop(columns=['Unnamed: 0','Unnamed: 0.1','Time to survival_cancer','Cancer_18month','Biopsy_18month','Cancer','Biopsy_18month','ID','RACE','HISPANIC','RUCA2','RUCA1','SVI','age_group','CCI','SVI_Person2','RUCA1_Person'])

Demographic_final['RPL_THEMES1'] = pd.qcut(Demographic_final['RPL_THEMES'], q=4, labels=False)
Demographic_final['SVI_Person1'] = pd.qcut(Demographic_final['SVI_Person'], q=4, labels=False)

Demographic_final = Demographic_final.drop(columns=['RPL_THEMES','SVI_Person','RUCA_Person','RUCA'])

Demographic_final = Demographic_final.dropna()

#VIF
# Demographic_final = Demographic6_18month.drop(columns=['Unnamed: 0','ID','RACE','HISPANIC','RUCA2','RUCA1','SVI','age_group','CCI','SVI_Person2','RUCA1_Person'])

def convert_to_dummies(dataframe, variables):
    for variable in variables:
        if  variable == 'Insurance':
            dummies = pd.get_dummies(dataframe[variable], prefix=variable)
            dummies = dummies.drop(columns=[f'{variable}_MEDICAID'])
        elif variable == 'RACE1':
            dummies = pd.get_dummies(dataframe[variable], prefix=variable)
            dummies = dummies.drop(columns=[f'{variable}_White'])
        elif variable == 'CCI1':
            dummies = pd.get_dummies(dataframe[variable], prefix=variable)
            dummies = dummies.drop(columns=[f'{variable}_None'])
        elif variable == 'RPL_THEMES1':
            dummies = pd.get_dummies(dataframe[variable], prefix=variable)
            dummies = dummies.drop(columns=[f'{variable}_3'])
        elif variable == 'SVI_Person1':
            dummies = pd.get_dummies(dataframe[variable], prefix=variable)
            dummies = dummies.drop(columns=[f'{variable}_3.0'])
        elif variable == 'age':
            continue  # Skip further processing for these variables
        else:
            dummies = pd.get_dummies(dataframe[variable], prefix=variable, drop_first=True)
        
        if variable != 'age':
            dummies = dummies.astype(int)
            dataframe = pd.concat([dataframe, dummies], axis=1)
            dataframe.drop(columns=[variable], inplace=True)
    
    return dataframe

Demographic_final_dummy = convert_to_dummies(Demographic_final, Dummy)
Demographic_final_dummy = Demographic_final_dummy.dropna()


# Split the data into training and testing sets
X = Demographic_final_dummy.drop(columns=['Time to survival_biopsy'])  # Input features
# Assuming 'Time to survival_biopsy' is non-negative
# Demographic_final_dummy['log_Time_to_survival_biopsy'] = np.log1p(Demographic_final_dummy['Time to survival_biopsy'])
# y = Demographic_final_dummy['log_Time_to_survival_biopsy'] 
y = Demographic_final_dummy['Time to survival_biopsy'] 

# Fit linear regression model
X = add_constant(X)  # Add constant term
model = sm.OLS(y, X)  # Ordinary Least Squares (OLS) regression
result = model.fit()

# Convert summary to a DataFrame
summary_df = pd.DataFrame(result.summary().tables[1])

# Save the summary DataFrame to a CSV file
# summary_df.to_csv('regression_model_summary3.csv', index=False)

# Display model summary
print(result.summary())

In [ ]:
Demographic7 = Demographic6_biopsy[['HISPANIC1','RACE1','SEX','Insurance','RUCA','SVI','age_group','CCI1','RUCA_Person','SVI_Person2']]

# Create an empty list to store tuples of (variable, level, count)
value_counts_list = []

# Iterate through each variable and calculate value_counts
for col in Demographic7:
    value_counts = pd.Series(Demographic7[col]).value_counts()
    for index, count in value_counts.items():
        value_counts_list.append((col, index, count))

# Convert the list of tuples into a DataFrame
value_counts_df = pd.DataFrame(value_counts_list, columns=['Variable', 'Level', 'Count'])
value_counts_df['Percent'] = value_counts_df['Count']/67238

#value_counts_df.to_csv('value_counts.csv')
value_counts_df

In [ ]:
def create_variable_dataframes(df, excel_filename):
    # Get the column names
    column_names = df.columns.tolist()
    column_names.remove('Time to survival_biopsy')
    
    # Create an empty DataFrame to store results
    result_df = pd.DataFrame()
    
    # Iterate through the variables of interest
    for column_name in column_names:
        # Create a new dataframe for the variable with the event and time-to-survival columns
        variable_df = pd.DataFrame({
            column_name: df[column_name],
            'Time to survival_biopsy': df['Time to survival_biopsy']
        })
        
        # Convert specific variables to dummy variables while dropping categories
        variables_to_convert = [column_name]  # Add more variables if needed
        Demographic_final_dummy = convert_to_dummies(variable_df, variables_to_convert)
        Demographic_final_dummy = Demographic_final_dummy.dropna()

        # Split the data into training and testing sets
        X = Demographic_final_dummy.drop(columns=['Time to survival_biopsy']) 
        y = Demographic_final_dummy['Time to survival_biopsy']
        X = add_constant(X)

        # Fit linear regression model
        model = sm.OLS(y, X)  # Change to OLS for linear regression
        result = model.fit()

        # Store model results in the master DataFrame
        model_summary = pd.DataFrame(result.summary().tables[1])
        model_summary['Variable'] = column_name  # Add a column for the variable name
        result_df = pd.concat([result_df, model_summary])
    
    # Save the master DataFrame to an Excel file
    result_df.to_excel(excel_filename, index=False)

# Call the function to create the single DataFrame with all model results
create_variable_dataframes(Demographic_final, 'linear_regression.xlsx')

## Biopsy to cancer

In [ ]:
Demographic6_18month1 = Demographic6_18month1.loc[Demographic6_18month1['Insurance'] != 'Other']
Demographic6_18month1 = Demographic6_18month1.loc[Demographic6_18month1['Insurance'] != 'NOPAY']
Demographic6_18month1 = Demographic6_18month1.loc[Demographic6_18month1['RACE1'] != 'Multiple race']

Demographic6_biopsy = Demographic6_18month1[Demographic6_18month1['Cancer_18month']  == 1]
Demographic_final = Demographic6_biopsy.drop(columns=['Unnamed: 0','Unnamed: 0.1','Time to survival_biopsy','Biopsy_18month','Cancer_18month','Cancer','ID','RACE','HISPANIC','RUCA2','RUCA1','SVI','age_group','CCI','SVI_Person2','RUCA1_Person'])

Demographic_final['RPL_THEMES1'] = pd.qcut(Demographic_final['RPL_THEMES'], q=4, labels=False)
Demographic_final['SVI_Person1'] = pd.qcut(Demographic_final['SVI_Person'], q=4, labels=False)

Demographic_final = Demographic_final.drop(columns=['RPL_THEMES','SVI_Person','RUCA_Person','RUCA'])

Demographic_final = Demographic_final.dropna()

#VIF
# Demographic_final = Demographic6_18month.drop(columns=['Unnamed: 0','ID','RACE','HISPANIC','RUCA2','RUCA1','SVI','age_group','CCI','SVI_Person2','RUCA1_Person'])

def convert_to_dummies(dataframe, variables):
    for variable in variables:
        if  variable == 'Insurance':
            dummies = pd.get_dummies(dataframe[variable], prefix=variable)
            dummies = dummies.drop(columns=[f'{variable}_MEDICAID'])
        elif variable == 'RACE1':
            dummies = pd.get_dummies(dataframe[variable], prefix=variable)
            dummies = dummies.drop(columns=[f'{variable}_White'])
        elif variable == 'CCI1':
            dummies = pd.get_dummies(dataframe[variable], prefix=variable)
            dummies = dummies.drop(columns=[f'{variable}_None'])
        elif variable == 'RPL_THEMES1':
            dummies = pd.get_dummies(dataframe[variable], prefix=variable)
            dummies = dummies.drop(columns=[f'{variable}_3'])
        elif variable == 'SVI_Person1':
            dummies = pd.get_dummies(dataframe[variable], prefix=variable)
            dummies = dummies.drop(columns=[f'{variable}_3.0'])
        elif variable == 'age':
            continue  # Skip further processing for these variables
        else:
            dummies = pd.get_dummies(dataframe[variable], prefix=variable, drop_first=True)
        
        if variable != 'age':
            dummies = dummies.astype(int)
            dataframe = pd.concat([dataframe, dummies], axis=1)
            dataframe.drop(columns=[variable], inplace=True)
    
    return dataframe

Demographic_final_dummy = convert_to_dummies(Demographic_final, Dummy)
Demographic_final_dummy = Demographic_final_dummy.dropna()

# Demographic6_18month2 = Demographic6_18month[Demographic6_18month['ID'].isin(biopsy_afterID)]
# Demographic_final = Demographic6_18month2.drop(columns=[ 'Time to survival_biopsy','Cancer_18month','Biopsy_18month','Cancer','Biopsy_18month','ID','RACE','HISPANIC','RUCA2','RUCA1','SVI','age_group','CCI','SVI_Person2','RUCA1_Person'])


# Split the data into training and testing sets
X = Demographic_final_dummy.drop(columns=['Time to survival_cancer'])  # Input features
# Demographic_final_dummy['log_Time_to_survival_cancer'] = np.log1p(Demographic_final_dummy['Time to survival_cancer'])

# y = Demographic_final_dummy['log_Time_to_survival_cancer'] 
y = Demographic_final_dummy['Time to survival_cancer']

# Fit linear regression model
X = add_constant(X)  # Add constant term
model = sm.OLS(y, X)  # Ordinary Least Squares (OLS) regression
result = model.fit()

# Convert summary to a DataFrame
summary_df = pd.DataFrame(result.summary().tables[1])

# Save the summary DataFrame to a CSV file
summary_df.to_csv('regression_model_summary4.csv', index=False)

# Display model summary
print(result.summary())

In [ ]:
Demographic7 = Demographic6_biopsy[['HISPANIC1','RACE1','SEX','Insurance','RUCA','SVI','age_group','CCI1','RUCA_Person','SVI_Person2']]

# Create an empty list to store tuples of (variable, level, count)
value_counts_list = []

# Iterate through each variable and calculate value_counts
for col in Demographic7:
    value_counts = pd.Series(Demographic7[col]).value_counts()
    for index, count in value_counts.items():
        value_counts_list.append((col, index, count))

# Convert the list of tuples into a DataFrame
value_counts_df = pd.DataFrame(value_counts_list, columns=['Variable', 'Level', 'Count'])
value_counts_df['Percent'] = value_counts_df['Count']/67238

#value_counts_df.to_csv('value_counts.csv')
value_counts_df

In [ ]:
def create_variable_dataframes(df, excel_filename):
    # Get the column names
    column_names = df.columns.tolist()
    column_names.remove('Time to survival_cancer')
    
    # Create an empty DataFrame to store results
    result_df = pd.DataFrame()
    
    # Iterate through the variables of interest
    for column_name in column_names:
        # Create a new dataframe for the variable with the event and time-to-survival columns
        variable_df = pd.DataFrame({
            column_name: df[column_name],
            'Time to survival_cancer': df['Time to survival_cancer']
        })
        
        # Convert specific variables to dummy variables while dropping categories
        variables_to_convert = [column_name]  # Add more variables if needed
        Demographic_final_dummy = convert_to_dummies(variable_df, variables_to_convert)
        Demographic_final_dummy = Demographic_final_dummy.dropna()

        # Split the data into training and testing sets
        X = Demographic_final_dummy.drop(columns=['Time to survival_cancer']) 
        y = Demographic_final_dummy['Time to survival_cancer']
        X = add_constant(X)

        # Fit linear regression model
        model = sm.OLS(y, X)  # Change to OLS for linear regression
        result = model.fit()

        # Store model results in the master DataFrame
        model_summary = pd.DataFrame(result.summary().tables[1])
        model_summary['Variable'] = column_name  # Add a column for the variable name
        result_df = pd.concat([result_df, model_summary])
    
    # Save the master DataFrame to an Excel file
    result_df.to_excel(excel_filename, index=False)

# Call the function to create the single DataFrame with all model results
create_variable_dataframes(Demographic_final, 'linear_regression2.xlsx')

# 7. Descriptive counts for the cohort ( The first step of the project)

In [ ]:
Demographic6['RACE'].value_counts()

In [ ]:
print(first_diag['age'].mean())
print(first_diag['age'].median())
print(first_diag['age'].std())
first_diag['age'].quantile([0.25, 0.5, 0.75])

In [ ]:
Demographic6['HISPANIC'].value_counts()

In [ ]:
Demographic6['SEX'].value_counts()

In [ ]:
Demographic6['Insurance'].value_counts()

In [ ]:
bin_edges = list(range(0, 80, 5))

plt.hist(first_date_rows1['ORIGINAL_BMI'], bins=bin_edges)  # Adjust the number of bins as needed

# Add labels and title
plt.xlabel('BMI')
plt.ylabel('Frequency')
plt.title('Histogram of BMI')
plt.xticks(bin_edges,rotation=45)
# Show the plot
plt.show()

In [ ]:
print(first_date_rows1['ORIGINAL_BMI'].mean())
print(first_date_rows1['ORIGINAL_BMI'].median())
print(first_date_rows1['ORIGINAL_BMI'].std())
first_date_rows1['ORIGINAL_BMI'].quantile([0.25, 0.5, 0.75])
print(first_date_rows1['ORIGINAL_BMI'].max())
BMI11 = first_date_rows1[first_date_rows1['ORIGINAL_BMI']<1000]
print(BMI11['ORIGINAL_BMI'].std())
BMI11['ORIGINAL_BMI'].max()

In [ ]:
print(Demographic6['RPL_THEMES'].mean())
print(Demographic6['RPL_THEMES'].median())
print(Demographic6['RPL_THEMES'].std())
Demographic6['RPL_THEMES'].quantile([0.25, 0.5, 0.75])

In [ ]:
Demographic6['RPL_THEMES'].dropna()

In [ ]:
print(Demographic6['SVI_Person'].mean())
print(Demographic6['SVI_Person'].median())
print(Demographic6['SVI_Person'].std())
Demographic6['SVI_Person'].quantile([0.25, 0.5, 0.75])

In [ ]:
Demographic6['SVI_Person'].dropna()

In [ ]:
Demographic6['CCI1'].value_counts()

In [ ]:
Demographic6['RUCA_Person'].value_counts()

In [ ]:
Demographic6['RUCA'].value_counts()

## For biopsy cohort

In [ ]:
# Demographic6_biopsy = Demographic6[Demographic6['ID'].isin(biopsy_afterID)]
Demographic6_biopsy = Demographic6_18month[Demographic6_18month['Biopsy_18month']  == 1]

In [ ]:
print(Demographic6_biopsy['age'].mean())
print(Demographic6_biopsy['age'].median())
print(Demographic6_biopsy['age'].std())
Demographic6_biopsy['age'].quantile([0.25, 0.5, 0.75])

In [ ]:
print(Demographic6_biopsy['SVI_Person'].mean())
print(Demographic6_biopsy['SVI_Person'].median())
print(Demographic6_biopsy['SVI_Person'].std())
Demographic6_biopsy['SVI_Person'].quantile([0.25, 0.5, 0.75])

In [ ]:
Demographic6_biopsy[Demographic6_biopsy['SVI_Person'].isna()] # RPL_THEMES=SVI for facility

In [ ]:
Demographic6_biopsy[Demographic6_biopsy['RPL_THEMES'].isna()] # RPL_THEMES=SVI for facility

In [ ]:
print(Demographic6_biopsy['RPL_THEMES'].mean())
print(Demographic6_biopsy['RPL_THEMES'].median())
print(Demographic6_biopsy['RPL_THEMES'].std())
Demographic6_biopsy['RPL_THEMES'].quantile([0.25, 0.5, 0.75])

In [ ]:
Demographic6_biopsy['Insurance'].value_counts()

In [ ]:
Demographic6_biopsy['SEX'].value_counts()

In [ ]:
Demographic6_biopsy['HISPANIC'].value_counts()

In [ ]:
Demographic6_biopsy['RACE'].value_counts()

In [ ]:
Demographic6_biopsy['RUCA'].value_counts() #RUCA = Rurality for facility

In [ ]:
Demographic6_biopsy

In [ ]:
Demographic6_biopsy['CCI1'].value_counts()

In [ ]:
Demographic6_biopsy['RUCA_Person'].value_counts() #Rurality for personal address

In [ ]:
BMI_biopsy = first_date_rows1[first_date_rows1['ID'].isin(biopsy_afterID)]
BMI_biopsy['BMI_group'].value_counts()

In [ ]:
print(BMI_biopsy['ORIGINAL_BMI'].mean())
print(BMI_biopsy['ORIGINAL_BMI'].median())
print(BMI_biopsy['ORIGINAL_BMI'].std())
BMI_biopsy['ORIGINAL_BMI'].quantile([0.25, 0.5, 0.75])

In [ ]:
BMI_biopsy['ORIGINAL_BMI'].max()

In [ ]:
BMI11 = BMI_biopsy[BMI_biopsy['ORIGINAL_BMI']<1000]
BMI11['ORIGINAL_BMI'].std()

In [ ]:
BMI11['ORIGINAL_BMI'].max()

## For cancer cohort

In [ ]:
# Demographic6_cancer = Demographic6[Demographic6['ID'].isin(cancer_afterID)]
Demographic6_cancer = Demographic6_18month1[Demographic6_18month1['Cancer_18month']  == 1]

In [ ]:
print(Demographic6_cancer['age'].mean())
print(Demographic6_cancer['age'].median())
print(Demographic6_cancer['age'].std())
Demographic6_cancer['age'].quantile([0.25, 0.5, 0.75])

In [ ]:
print(Demographic6_cancer['SVI_Person'].mean())
print(Demographic6_cancer['SVI_Person'].median())
print(Demographic6_cancer['SVI_Person'].std())
Demographic6_cancer['SVI_Person'].quantile([0.25, 0.5, 0.75])

In [ ]:
Demographic6_cancer[Demographic6_cancer['SVI_Person'].isna()]

In [ ]:
print(Demographic6_cancer['RPL_THEMES'].mean())
print(Demographic6_cancer['RPL_THEMES'].median())
print(Demographic6_cancer['RPL_THEMES'].std())
Demographic6_cancer['RPL_THEMES'].quantile([0.25, 0.5, 0.75])

In [ ]:
Demographic6_cancer[Demographic6_cancer['RPL_THEMES'].isna()]

In [ ]:
Demographic6_cancer['Insurance'].value_counts()

In [ ]:
Demographic6_cancer['SEX'].value_counts()

In [ ]:
Demographic6_cancer['HISPANIC'].value_counts()

In [ ]:
Demographic6_cancer['RACE'].value_counts()

In [ ]:
Demographic6_cancer['RUCA'].value_counts()

In [ ]:
Demographic6_cancer['CCI1'].value_counts()

In [ ]:
Demographic6_cancer['RUCA_Person'].value_counts()

In [ ]:
BMI_biopsy = first_date_rows1[first_date_rows1['ID'].isin(cancer_afterID)]
BMI_biopsy['BMI_group'].value_counts()

In [ ]:
print(BMI_biopsy['ORIGINAL_BMI'].mean())
print(BMI_biopsy['ORIGINAL_BMI'].median())
print(BMI_biopsy['ORIGINAL_BMI'].std())
BMI_biopsy['ORIGINAL_BMI'].quantile([0.25, 0.5, 0.75])

In [ ]:
BMI_biopsy['ORIGINAL_BMI'].max()